In [1]:
import os
import glob
import time
import numpy as np
import scipy.signal
import pandas as pd
from scipy.signal import savgol_filter
from scipy.signal import find_peaks
import warnings
import json
from pandas.io.json import json_normalize
from scipy.interpolate import interp1d
import sys

In [2]:
#INPUT FILE RUN
#Cambiar al archivo que se quiere usar
#input_file='Inputdata_Lectura.ipynb'
input_file='Inputdata_TMTA.ipynb'

#Obtiene directorio actual
cwd = os.getcwd()
#Busca directorio arriba
here=os.path.dirname(cwd)
#Arma el directorio donde están los inputs
inputs=os.path.join(here,'INPUTS')
#Arma el camino al archivo
inputs=os.path.join(inputs,input_file)

print('\033[91m' + 'Archivo Input:' + '\033[0m' + inputs)

%run $inputs

Archivo Input:C:\Ana\Python\Codigos\INPUTS\Inputdata_TMTA.ipynb


In [3]:
#Archivos donde están las funciones que llama el main
%run IVTNH.ipynb
%run ReadWrite.ipynb
%run datainterp.ipynb

In [4]:
#Busca en el directorio de entrada los archivos con estas extensiones
files1=glob.glob(directory_in + '*.csv')
files1.extend(glob.glob(directory_in + '*.psimesh'))
files1.extend(glob.glob(directory_in + '*.json'))

kmax=len(files1)
print('\033[91m' + 'Cantidad de archivos que encontró:' + '\033[0m',kmax)
print('\033[91m' + 'Nombres de los archivos:' + '\033[0m',files1)

#Para guardar los umbrales de cada sujeto del algoritmo de velocidad
umbrales = np.zeros(kmax,dtype={"names":('k','umbral'),'formats':('int','float')})

#Loop en cada uno de los archivos en el directorio de entrada
k=0
for file in files1:
    filename_in=file
    print('\033[91m' + 'Procesando archivo:' + '\033[0m',filename_in)
    filename1, file_extension = os.path.splitext(filename_in)

    #Contador de tiempo
    tic = time.perf_counter()
    
    #Lee el archivo de acuerdo a su extensión
    if file_extension=='.csv':
        df=read_csv(filename_in,init,fin)
    else:
        df=read_json(k,filename_in,resx,resy,file_extension,Fs)

    #Filtro SV en todas las coordenadas (x,y,r) y saca la velocidad en la coordenada elegida
    df_int=data_interp(df,Fs)
    del df
    
    #Método I-VT (velocidad) modificado umbral de Nystrom-Holmvquist.
    (saccades_frame,fix_frame,data_frame,umbral)=ivt_NH(df_int,Fs,k)
    if saccades_frame["onset"].size<=10:
        k=k+1
        continue
    
    umbrales["umbral"][k]=umbral
    umbrales["k"][k]=k
    
    #Nombre archivos de salida. Son el mismo sujeto, con sacada o data como prefijo para identificar los datos.
    file1=os.path.split(file)
    file2=os.path.splitext(file1[1])
    file_out_saccade=os.path.join(directory_out, "sacadas" + "-IVTNH" + "_" + file2[0] + ".csv")
    file_out_data=os.path.join(directory_out, "datos" + "-IVTNH" + "_" + file2[0] + ".csv")
    file_out_fix=os.path.join(directory_out, "fix" + "-IVTNH" + "_" + file2[0] + ".csv")
    
    #Imprime a un csv las sacadas y datos.
    print(file_out_saccade,file_out_fix,file_out_data)
    print_to_csv(saccades_frame,fix_frame,data_frame,file_out_saccade,file_out_fix,file_out_data)
    toc = time.perf_counter()
    
    print('\033[91m' + f"For file {k} with IVT-NH {toc - tic:0.4f} seconds"  + '\033[0m')
    k=k+1
    if k==kmax:
        umb_frame=pd.DataFrame(umbrales)
        file_out_umb=os.path.join(directory_out,"umbrales.csv")
        umb_frame.to_csv (file_out_umb,  header=True, index=False)
        break
        
    del filename_in
    del df_int
    del saccades_frame
    del fix_frame
    del data_frame
    del file_out_saccade
    del file_out_fix
    del file_out_data

Cantidad de archivos que encontró: 26
Nombres de los archivos: ['C:\\Ana\\Python\\Data\\Tobii\\TMTA\\TMTA_h20_485_310718.json', 'C:\\Ana\\Python\\Data\\Tobii\\TMTA\\TMTA_h21_087_190718.json', 'C:\\Ana\\Python\\Data\\Tobii\\TMTA\\TMTA_h21_184_060718.json', 'C:\\Ana\\Python\\Data\\Tobii\\TMTA\\TMTA_h21_382_160718.json', 'C:\\Ana\\Python\\Data\\Tobii\\TMTA\\TMTA_h21_455_030718.json', 'C:\\Ana\\Python\\Data\\Tobii\\TMTA\\TMTA_h21_685_160718.json', 'C:\\Ana\\Python\\Data\\Tobii\\TMTA\\TMTA_h21_689_160718.json', 'C:\\Ana\\Python\\Data\\Tobii\\TMTA\\TMTA_h21_939_050718.json', 'C:\\Ana\\Python\\Data\\Tobii\\TMTA\\TMTA_h22_951_310718.json', 'C:\\Ana\\Python\\Data\\Tobii\\TMTA\\TMTA_h24_371_060718.json', 'C:\\Ana\\Python\\Data\\Tobii\\TMTA\\TMTA_h26_295_190718.json', 'C:\\Ana\\Python\\Data\\Tobii\\TMTA\\TMTA_h27_188_020718.json', 'C:\\Ana\\Python\\Data\\Tobii\\TMTA\\TMTA_h29_208_060718.json', 'C:\\Ana\\Python\\Data\\Tobii\\TMTA\\TMTA_h30_138_190718.json', 'C:\\Ana\\Python\\Data\\Tobii\\TMTA\\TMT

Cantidad de sacadas en pos-procesamiento: 128
C:\Ana\Python\Resultados\TMT\Modelo2D\sacadas-IVTNH_TMTA_h37_958_030718.csv C:\Ana\Python\Resultados\TMT\Modelo2D\fix-IVTNH_TMTA_h37_958_030718.csv C:\Ana\Python\Resultados\TMT\Modelo2D\datos-IVTNH_TMTA_h37_958_030718.csv
For file 14 with IVT-NH 0.6537 seconds
Procesando archivo: C:\Ana\Python\Data\Tobii\TMTA\TMTA_m20_187_160718.json
Umbral velocidad: 924.8718711523636
Cantidad de sacadas en pos-procesamiento: 125
C:\Ana\Python\Resultados\TMT\Modelo2D\sacadas-IVTNH_TMTA_m20_187_160718.csv C:\Ana\Python\Resultados\TMT\Modelo2D\fix-IVTNH_TMTA_m20_187_160718.csv C:\Ana\Python\Resultados\TMT\Modelo2D\datos-IVTNH_TMTA_m20_187_160718.csv
For file 15 with IVT-NH 0.7015 seconds
Procesando archivo: C:\Ana\Python\Data\Tobii\TMTA\TMTA_m20_283_190718.json
Umbral velocidad: 612.1094232347449
Cantidad de sacadas en pos-procesamiento: 81
C:\Ana\Python\Resultados\TMT\Modelo2D\sacadas-IVTNH_TMTA_m20_283_190718.csv C:\Ana\Python\Resultados\TMT\Modelo2D\fix-I